# DeiT: Data-efficient Image Transformers
**Transformers go brum brum**

Hi guys! Today we are going to implement [Training data-efficient image transformers & distillation through attention](https://arxiv.org/abs/2012.12877) a new method to perform knoledge distiallion on Vision Transformers called DeiT.

You will soon see how elegant and simple this new approach is.


Code is [here](https://github.com/FrancescoSaverioZuppichini/DeiT), an interactive version of this article can be downloaded from [here[(https://github.com/FrancescoSaverioZuppichini/DeiT/blob/main/README.ipynb).

DeiT is available on my new computer vision library called [glasses](https://github.com/FrancescoSaverioZuppichini/glasses)

Before starting I **highly** reccomand to first have a look at [Vision Transformers](https://towardsdatascience.com/implementing-visualttransformer-in-pytorch-184f9f16f632)

## Attention Distillation

So, let's first have a look at the performance

![alt](DeiTTable)